In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import sys
sys.path.append("..")

In [2]:
from utils.helper import get_encoding_data

In [3]:
X, y, X_test = get_encoding_data(fmri_dir="../data/participants_data_v2021",
                                 activations_dir='../data/features/alexnet/pca_100',
                                 layer_id="layer_1",
                                 subject_id="sub01",
                                 roi_type="V1")
y.shape

(1000, 232)

In [6]:
from encoding_models.ols import fit_linear_model, predict_linear_model
import numpy as np
model_config = {"lambda_reg": 0.0}
model_params = fit_linear_model(model_config, X, y)
y_pred = predict_linear_model(model_params, X)
np.mean((y-y_pred)**2)

DeviceArray(0.20087245, dtype=float32)

In [ ]:
import torch
import torch.nn as nn
from encoding_models.networks.body_builder import BodyBuilder
from encoding_models.networks.train_dojo import TrainDojo
from encoding_models.networks.data_loader import get_train_test_split_loaders
from encoding_models.networks.optimizer import set_optimizer

net_config = {"input_dim": (1, X.shape[1]),
              "layers_info": [["flatten"],
                              ["linear", 256, True],
                              ["linear", 256, True],
                              ["linear", 256, True],
                              ["linear", y.shape[1], True]],
              "output_act": "identity",
              "hidden_act": "relu",
              "dropout": 0.0,
              "batch_norm": False}

network = BodyBuilder(**net_config)
optimizer = set_optimizer(network, "Adam", 3e-4)
criterion = nn.MSELoss()
train_loader, test_loader = get_train_test_split_loaders(X, y, 0.1, batch_size=32)

In [ ]:
dojo = TrainDojo(network, optimizer, criterion,
                 train_loader, test_loader,
                 log_batch_interval=10)

In [ ]:
dojo.train(15)